# Text Analytics Lab 1: Regular Expressions and Classifiers

### Learning Outcomes
* Be able to set up a Python and Jupyter notebook environment for text analytics
* Understand how to use regular expressions to preprocess text
* Know how to carry out text normalisation and train and test naïve Bayes and logistic regression classifiers.
* Be able to examine learned model parameters and explain how each classifier makes a decision.

### Outline

1. Getting started: how to set up your environment, Jupyter notebooks introduction
1. Acquiring raw text data
1. Regular expressions
1. Text normalisation 
1. Training and evaluating naïve Bayes using Scikit-learn.
1. Training and evaluating logistic regression using Scikit-learn.
1. Optional extension: lemmatization and bigram features.
1. Optional extensions: lexicon features.

### How To Complete This Lab

Read the text and the code then look for 'TODOs' that instruct you to complete some missing code. Look out for 'QUESTIONS' which you should try to answer before moving on to the next cell. Aim to work through the lab during the scheduled lab hours. To get help, you can talk to TAs or the lecturer during the labs, post questions to the Blackboard discussion board or on Teams, or ask a question in the lectures. 

The labs *will not be marked*. However, they will prepare you for the coursework, so try to keep up with the weekly labs and have fun with the exercises! To understand what's going on inside the methods we use here, make sure to watch the lecture videos for the same week.

## 1. Getting Started

### Setting up your environment

We recommend using ```conda``` to create an environment with the correct versions of all the packages you need for these labs. You can install either Anaconda or Miniconda, which will include the ```conda``` program. 

We provide a .yml file that lists all the packages you will need, and the versions that we have tested the labs with. You can use this file to create your environment as follows.

1. Open a terminal. Use the command line to navigate to the directory containing this notebook and the file ```crossplatform_environment.yml```. You can use the command ```cd``` to change directory on the command line.

1. For Lab machines in MVB 2.11 only: Load the Anaconda module: ```module load anaconda/3-2024```.

1. Run the conda program by typing ```conda env create -f crossplatform_environment.yml```, then answer any questions that appear on the command line.

1. Activate the environment by running the command ```conda activate text_analytics```.

1. Make kernel available in Jupyter: ```python -m ipykernel install --user --name=text_analytics```.

1. Relaunch Jupyter: shutdown any running instances, and then type ```jupyter lab``` into your command line.

1. Find this notebook and open it up again.

1. Go to the top menu and change the kernel: click on 'Kernel'--> 'Change kernel' --> text_analytics.

You should now be ready to go!

The core libraries we will be using in this unit are:

- [Datasets](https://huggingface.co/docs/datasets/), produced by HuggingFace, is a hub for lots of interesting text datasets.
- [NLTK](https://www.nltk.org), a comprehensive NLP library.
- [Scikit-learn](https://scikit-learn.org/stable/user_guide.html), for machine learning and classifier evaluation.
- [Gensim](https://radimrehurek.com/gensim/), for topic modelling.
- [Transformers](https://huggingface.co/docs/transformers/en/index), for state-of-the-art NLP models. 
- [PyTorch](https://pytorch.org/), a framework for deep learning. 

The libraries above have good documentation, which is available either online (links above) or via Python itself, e.g. `help(numpy.array)` in the Python interpreter. 

### Refreshers for Python and Jupyter

**Skip this part if you're already familiar with Python and Jupyter notebooks.**

This lab assumes you have used Python and Jupyter Notebooks before. 

For an introduction or refresher on Python, see the [Introduction to Python lab](https://github.com/UoB-COMS21202/lab_sheets_public/tree/master/lab_1) or the University of Bristol [Beginning Python](https://milliams.gitlab.io/beginning_python/) course. If you are a beginner with Python, you might also like to look at Chapter 1 in the NLTK book, which also provides a guide for "getting started with Python": https://www.nltk.org/book/ 

The labs will be run on [Jupyter Notebook](http://jupyter.org/), an interactive coding environment embedded in a webpage supporting various programing languages (Python, R, Lua, etc.) through the concept of kernels.  

It allows you to enrich your code with complex comments formatted in Markdown and $\LaTeX$, as well as to place the results of your computation right below your code.

Notebooks are organised in cells which can contain either code (in our case, this will be Python code) or text, which can be easily and nicely formatted using the Markdown notation. 

To edit an already existing cell simply double-click on it. You can use the toolbar to insert new cells, edit and delete them (or use keyboard shortcuts which are very handy to speed up coding). 

Cells can be run by hitting `shift+enter` when editing a cell or by clicking on the `Run` button at the top. Running a Markdown cell will simply display the formatted text, while running a code cell will execute the commands executed in it. Create new cells with the keyboard shortcut `esc` followed by `A` or `B`.

**Note**: when you run a code cell, all the created variables, implemented functions and imported libraries will be then available to every other code cell. It is commonly assumed that cells will be run in the correct sequence and running them repeatedly or out-of-order may sometimes cause errors. To reset all variables and functions (for debugging) simply click `Kernel > Restart` from the Jupyter menu.

#### Markdown 

Markdown cells allow you to write fancy and simple comments: all of this is written in Markdown - double click on this cell to see the source. An introduction to Markdown syntax can be found [here](https://daringfireball.net/projects/markdown/syntax).

As Markdown is translated to HTML upon displaying it also allows you to use pure HTML: more details are available [here](https://daringfireball.net/projects/markdown/syntax#html).

Finally, you can also display simple $\LaTeX$ equations in Markdown thanks to `MathJax` support. For inline equations wrap your equation between `$` symbols; for display mode equations use `$$`.

## 1. Acquiring Raw Text Data

Now, let's get some text data! We'll start with the IMDB dataset, which contains movie reviews along with their classification into "positive" or "negative" sentiment. Run the code below to download the data from [HuggingFace's datasets hub](https://huggingface.co/datasets/imdb):

In [76]:
from datasets import load_dataset
import numpy as np

cache_dir = "./data_cache"

# The data is already divided into training and test sets.
# Load the training set:
train_dataset = load_dataset(
    "imdb", # name of the dataset collection
    split="train",  # train or test
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

train_dataset = np.random.choice(train_dataset, 5000, replace=False)  # we'll only use a subset of the data in this lab so that the code runs quicker


Training dataset with 25000 instances loaded


We can access the documents in the dataset like elements in a list. For example, document 3 looks like this:

In [77]:
train_dataset[10]

{'text': "I want to state first that I am a Christian (and that I do work in the film and TV industry) so I understand what it is like to work on a feature length film so props to the filmmakers in that regard. I'm all for positive, uplifting messages if they are true to the nature of life (that this is a fallen world and that things don't always work out ... even for followers of Christ). I'm glad that others are having such overwhelmingly positive reactions to the overt Christian message; for me it was just that the execution is where the film fell on its face. A movie lives and dies on its story and here you have one dimensional stereotypes, exposition aplenty, and spontaneous changes in character behavior that are inexplicably to say the least. I believe that a film does not have to club you over the head with its message to get the point across. I'm sure the Kendrick bros. will improve with time and that their storytelling methods will as well. Maybe they could direct someone else

# 2. Regular Expressions

## 2.1 Search

We'll start by trying out some regular expressions. Suppose we want to identify tweets where people discuss really loved about certain movies. We could start by looking for tweets that contain the word 'love'. A first step would be to find all occurrences of the word 'love'. Review the code below to see how we can do this:

In [78]:
import re  # Python regular expressions library

all_matches = []

for review in train_dataset:
    matches = re.findall('love', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

1857
1857
love


This has given us a list of matches in the variable `all_matches`, which all contain the string 'love', but not the sentences themselves.
This isn't very useful, but we can do better if we define the right regular expression!

Regular expressions define a pattern, rather than a specific string, allowing us to generalise our search and retrieve a many different strings that match the pattern.
In Python, we differentiate a regular expression from a normal string by putting an 'r' character in front of the string.

We can generalise our search by using a _disjunction_, which will match against any one of a set of characters. The disjunction is written inside square brackets. 

Let's try to retrieve instances of the word "love" followed by any letter. We can write a disjunction that matches any lower case letter as `[a-z]`:

In [79]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'love [a-z]', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

924
924
love l
love a
love t
love h
love k
love p
love s
love v
love u
love o
love z
love y
love f
love i
love b
love m
love w
love q
love g
love c
love j
love e
love n
love r
love d


Our current search only matches a single letter of the word after 'love'. The length of that following word is variable, so how can we write an expression to match the whole word? 

Here, we can use a special character, '\*', which will match against zero or more repetitions of the preceding regular expression. Let's try it out:

In [80]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'love [a-z]*', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

1018
1018
love at
love just
love many
love or
love dramas
love lord
love are
love one
love hate
love begins
love since
love our
love where
love you
love it
love justify
love any
love must
love animated
love birds
love the
love from
love itself
love as
love would
love stories
love laughing
love indy
love terrible
love talk
love transcends
love seems
love special
love interest
love such
love affair
love puppet
love toward
love movies
love however
love when
love their
love surfing
love bad
love women
love blooming
love in
love most
love rival
love music
love truth
love zombie
love beyond
love these
love a
love triumphs
love obsession
love turning
love being
love to
love your
love she
love and
love both
love affairs
love hearing
love ones
love slow
love is
love scene
love me
love cheating
love wins
love like
love scenes
love relationship
love quirky
love sick
love every
love all
love while
love vs
love subplot
love with
love animals
love mystery
love life
love crappy
love goddesses
love hu

Let's say we only want to retrieve the word following 'love', not the string containing 'love ' itself. 
We can do this using parentheses to create _groups_ of characters, such as this: `([a-z]*)`. The resulting matches will be returned as tuples of groups, and any characters not inside parentheses will not be returned as part of any group. Try out the code below to see this, and note that the space character after 'love' is not returned in the matches.

In [81]:
all_matches = []

for review in train_dataset:
    matches = re.findall(r'(love) ([a-z]*)', review['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

for match in set(all_matches):  # just print the following
    print(match[1]) 

1018
1018
('love', 'shifts')
('love', 'rival')
('love', 'crucifies')
('love', 'already')
('love', 'they')
('love', 'my')
('love', 'each')
('love', 'problem')
('love', 'has')
('love', 'triumphs')
('love', 'theme')
('love', 'would')
('love', 'such')
('love', 'killer')
('love', 'watching')
('love', 'any')
('love', 'as')
('love', 'musicals')
('love', 'upon')
('love', 'animals')
('love', 'you')
('love', 'song')
('love', 'birds')
('love', 'indy')
('love', 'first')
('love', 'wins')
('love', 'begins')
('love', 'when')
('love', 'she')
('love', 'yourself')
('love', 'like')
('love', 'corny')
('love', 'slow')
('love', 'women')
('love', 'mystery')
('love', 'absolutely')
('love', 'cheating')
('love', 'subplot')
('love', 'surfing')
('love', 'some')
('love', 'things')
('love', 'of')
('love', 'goddesses')
('love', 'it')
('love', 'another')
('love', 'all')
('love', 'able')
('love', 'between')
('love', 'seems')
('love', 'he')
('love', 'every')
('love', 'scene')
('love', 'terrible')
('love', 'animated')
(

Now, let's try to retrieve the preceding words as well. It would be better to match capital letters as well as lower case, which we can do with the disjunction `[a-zA-Z]`. 

TODO 1: complete the code below to retrieve only the words that precede and follow 'love', including capitalised and lower case words.

In [82]:
all_matches = []

for review in train_dataset:
    
    ### WRITE YOUR CODE HERE
    matches = re.findall(r"like [a-z]*", review['text'])
    ########
    
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

4075
4075
like adult
like insects
like dinosaur
like inane
like sex
like remember
like ghosts
like instead
like busted
like mentally
like pace
like receiving
like about
like evolving
like people
like outfit
like ten
like listening
like losing
like amateur
like story
like shes
like almost
like junior
like complete
like marmite
like crap
like family
like shedding
like wearing
like eyes
like beginners
like dummies
like nobody
like action
like therapies
like walnuts
like was
like puritans
like rank
like they
like anyone
like malaria
like musicals
like toasters
like reading
like scenery
like high
like snakes
like than
like unfinished
like us
like names
like gone
like dimensions
like place
like nor
like contro
like mannequins
like elite
like parts
like by
like performance
like special
like puking
like bosom
like letting
like coffee
like crazy
like described
like fingernails
like period
like cutting
like moi
like contemplating
like only
like dags
like but
like quiet
like re
like pornographic


This is starting to look more useful, but we still want to retrieve whole sentences. 

Sentences in English are usually demarcated by punctuation, so let's use the following punctuation marks to identify sentence boundaries: '.', '!', '?'. Those punctuation marks are special characters when used in regular expressions, so to force Python to interpret them literally, we need to put the escape character '\\' in front of them. 

Now, we can write a disjunction that matches against the punctuation like this: `[\.\!\?]`.

So far, we have assumed the text consists only of letters. Can you think of any characters we have excluded here? 

A better way to find all matches would be to use _negation_ to match against any character _except_ the punctuation marks that bound the sentences. A negation will match any character except those specified, which we can write like this: `[^\.\!\?]`, where the '^' indicates the negation.


TODO 2: Retrieve whole sentences containing 'love'. To do this, modify our previous expression by using negation to match all of the characters except '.', '!', and '?'.

In [83]:
all_matches = []

for review in train_dataset:
    
    ### WRITE YOUR CODE HERE
    matches = re.findall(r'love [^\.\!\?]*', review['text'])
    ########
    
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match)  

974
love with Elizabeth Taylor
love it because of the opportunity it affords to say, "Boys are so weird
love corny romances and give them plenty of space but really---this is unbelievable
love for its writer-director, Jonathan Betuel (who also served as one of the producers); sadly, the end results are anemic, to be charitable
love this one also
love story that takes place during the ill-fated voyage on the unsinkable ship
love the original movie, I read the book and I love it
love song to the bull "Dominic"
love story about two cons
love her overwhelming refugee husband and the problematic but forceful plans of Goldblum
love laughing during horror movies (Return of the Living Dead 2, Evil Dead), but I don't know if we were supposed to be laughing here
love are themes that are all extensively painted with music, close-ups and silent scenes like telling a poem
love to see this released someday to a DVD, PLEASE RELEASE IT, whoever's concerned
love the actors who have played the character

Look at the results -- does the regular expression correctly return sentences containing 'love'?

There are lots more special characters that you can use to form really powerful regular expressions for segmenting, retrieving and substituting text. For your reference, you can find a complete list [here](https://docs.python.org/3/library/re.html#regular-expression-syntax).

## 2.2 Substitution

We can also use regular expressions during _preprocessing_ to clean up text and prepare it for further analysis. For this, we use regular expression _substitution_, which finds a matching string within a larger piece of text, and replaces it with another string.

Let's use this to clean up the text by removing the line break characters.

In Python, we can use the re.sub() function, which takes three arguments:
1. The expression to match. 
2. The pattern we should replace it with
3. The text to apply the subtitution to. 

Some of the reviews contain some HTML formatting code, `<br />`, which we can try to remove to clean up the text. We can do this by writing an expression for the first argument of re.sub() that matches '<br />'. Take a look at how this works by running the code below:

In [84]:
print('ORIGINAL TEXT: ')
print(train_dataset[0]['text'])
    
clean_article = re.sub(r'<br />', r' ', train_dataset[0]['text'])  # replace HTML breaks with a space
    
print('CLEANER TEXT: ')
print(clean_article)

ORIGINAL TEXT: 
Yokai Monsters: Spook Warfare (Yokai daisenso, 2005) a movie about "yokai" or traditional Japanese "monsters" of folklore. It is alternatively known as Big Monster War or as Ghosts on Parade.<br /><br />The yokai of the first installment include the teapot freak, kappa water imp, a living 'brella, a woman whose sheeks can grow extremely gigantic, a woman with a second face on the back of her head, a dwarf priest with an enormous gourd-like wrist, & so on.<br /><br />These sorts of whimsical monsters derive not only from fairy lore, but from a type of summer entertainment of the Tokugawa Era, comparable to today's Halloween haunted houses, or the "freak shows" of yesteryear but with exclusively phony freaks. Ghosts & goldfish monsters & dancing one-headed umbrellas were trumped up to create "chills" during the hot summers. The fatcheek woman & such were recreated by tricks or illusions, based on monsters depicted in medieval scrolls; & if their design for the movie is a 

# 3. Text Normalisation 

For most text analytics tasks, we will first need to transform the raw text to a suitable format for input to method such as a classifier. This process is called _text normalisation_ and is part of the _preprocessing_ stage. There are three common steps:

1. Sentence segmentation: we have already tried out a basic approach to obtaining complete sentences using regular expressions. This would need to be modified to return a list of all sentences in a document. 
2. Tokenisation, in which the sentences are split into a sequence of tokens, which include words, numbers and punctuation marks.
3. Word normalisation, in which different forms of a word are replaced by a root form.

We are now going to see how to perform these steps using the NLTK library.

## 3.1 Sentence Segmentation

Let's start by using NLTK to split a document into sentences. This should give better results than our regular expressions above.

You may get some errors from NLTK when you try to use sent_tokenize or word_tokenize further down. This is usually because you need to download and install some NLTK data. Please check the error message to find out which package is required. You probably need to install packages called 'punkt' and 'wordnet'. You can install these packages by running the cell below.

In [85]:
import nltk 

# nltk.download('punkt_tab')


nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/sisjkly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sisjkly/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
import nltk

review = train_dataset[3]['text']

sents = nltk.sent_tokenize(review)

for sent in sents[:5]:
    print("<SENTENCE>")
    print(sent)  # print the first five sentences of this document

<SENTENCE>
Really, when it comes down to it, this movie is just not funny.
<SENTENCE>
Josh Kornbluth is the antithesis of funny, and yet he thinks he's hilarious.
<SENTENCE>
The plot about mailing the 17 letters was fine at first, until Josh's "look at me, I'm wacky" approach began infesting each and every scene.
<SENTENCE>
The Judas Priest joke may have been the least funny thing I have ever been privy to, and that is saying quite a bit, since it does have stiff competition from all the other jokes in the movie.
<SENTENCE>
And as for the incredible overuse of the Pixies' "Debaser," I have just one thing to say: What did the Pixies ever do to deserve a fate like this?


TODO 3: Use the regular expression substitution code from section 2.2 to remove the '\<br /\>' tags from the sentences displayed above and print the results.

In [87]:
clean_sents = []

for sent in sents[:5]:
    
    ### WRITE YOUR OWN CODE HERE
    sent = re.sub(r'<br />', r' ', sent)  # replace HTML breaks with a space
    #######
    
    print("<SENTENCE>")
    print(sent)  # print the first five sentences of this document
    
    clean_sents.append(sent)  # save the cleaned sentences for later

<SENTENCE>
Really, when it comes down to it, this movie is just not funny.
<SENTENCE>
Josh Kornbluth is the antithesis of funny, and yet he thinks he's hilarious.
<SENTENCE>
The plot about mailing the 17 letters was fine at first, until Josh's "look at me, I'm wacky" approach began infesting each and every scene.
<SENTENCE>
The Judas Priest joke may have been the least funny thing I have ever been privy to, and that is saying quite a bit, since it does have stiff competition from all the other jokes in the movie.
<SENTENCE>
And as for the incredible overuse of the Pixies' "Debaser," I have just one thing to say: What did the Pixies ever do to deserve a fate like this?


## 3.2 Tokenisation

NLTK provides a similar function for tokenizing the text at the word level. You can find the documentation [here](https://www.nltk.org/api/nltk.tokenize.html). Most tokenizers use either regular expressions or a machine learning model that was trained on a large dataset to learn token-splitting rules. 

TODO 4: Use word_tokenize() to tokenize each of the sentences from the last cell.

In [88]:
from nltk.tokenize import word_tokenize
tokenized_sents = []

for sent in clean_sents:
    ### WRITE YOUR OWN CODE HERE
    tokens = word_tokenize(sent)
    #######
    
    print("<TOKENS>")
    print(tokens)
    
    tokenized_sents.append(tokens)

<TOKENS>
['Really', ',', 'when', 'it', 'comes', 'down', 'to', 'it', ',', 'this', 'movie', 'is', 'just', 'not', 'funny', '.']
<TOKENS>
['Josh', 'Kornbluth', 'is', 'the', 'antithesis', 'of', 'funny', ',', 'and', 'yet', 'he', 'thinks', 'he', "'s", 'hilarious', '.']
<TOKENS>
['The', 'plot', 'about', 'mailing', 'the', '17', 'letters', 'was', 'fine', 'at', 'first', ',', 'until', 'Josh', "'s", '``', 'look', 'at', 'me', ',', 'I', "'m", 'wacky', "''", 'approach', 'began', 'infesting', 'each', 'and', 'every', 'scene', '.']
<TOKENS>
['The', 'Judas', 'Priest', 'joke', 'may', 'have', 'been', 'the', 'least', 'funny', 'thing', 'I', 'have', 'ever', 'been', 'privy', 'to', ',', 'and', 'that', 'is', 'saying', 'quite', 'a', 'bit', ',', 'since', 'it', 'does', 'have', 'stiff', 'competition', 'from', 'all', 'the', 'other', 'jokes', 'in', 'the', 'movie', '.']
<TOKENS>
['And', 'as', 'for', 'the', 'incredible', 'overuse', 'of', 'the', 'Pixies', "'", '``', 'Debaser', ',', "''", 'I', 'have', 'just', 'one', 'thing

Run the code below to see how NLTK has handled the non-letter characters. 
* What does it do with most punctuation marks? 
* When does it not split tokens based on punctuation?

In [89]:
for sent in tokenized_sents:
    for tok in sent:
        if re.search(r'[^a-zA-Z0-9]', tok):  # find the non-letter and non-digit characters
            print(tok)  # print the entire token containing the non-letter/non-digit character

,
,
.
,
's
.
,
's
``
,
'm
''
.
,
,
.
'
``
,
''
:
?


## 3.3 Word Normalisation

Many words can appear in different forms, including: 
* Conjugated verbs
* Plural and singular nouns
* Common abbrevations and synonyms like "USA" and "US". 

Mapping all of these surface forms to a single root form reduces the size of the vocabulary that we have to deal with and can therefore improve the performance of text classifiers or topic models.

The two most widely used tools for this task in English are the Porter Stemmer and WordNet Lemmatizer. These tools apply a series of regular expression substitutions to tokenised text to convert words to a standard format. 
* The Porter stemmer is much faster but just removes word prefixes and endings, which leads to some errors. It is often used when real-time or high-volume text processing is needed.
* As well as applying regular expressions, lemmatizers look words up in a dictionary to find their root forms, so are more accurate but much slower. 

Let's start by applying the [Porter Stemmer class](https://www.nltk.org/_modules/nltk/stem/porter.html) to our tokenised text by calling the stem() method:

In [90]:
stemmer = nltk.PorterStemmer() 
stemmed_sents = []

for sent in tokenized_sents:
    stemmed_sent = [stemmer.stem(tok) for tok in sent]
    
    stemmed_sents.append(stemmed_sent)
    
    print("<STEMMED TOKENS>")
    print(stemmed_sent)

<STEMMED TOKENS>
['realli', ',', 'when', 'it', 'come', 'down', 'to', 'it', ',', 'thi', 'movi', 'is', 'just', 'not', 'funni', '.']
<STEMMED TOKENS>
['josh', 'kornbluth', 'is', 'the', 'antithesi', 'of', 'funni', ',', 'and', 'yet', 'he', 'think', 'he', "'s", 'hilari', '.']
<STEMMED TOKENS>
['the', 'plot', 'about', 'mail', 'the', '17', 'letter', 'wa', 'fine', 'at', 'first', ',', 'until', 'josh', "'s", '``', 'look', 'at', 'me', ',', 'i', "'m", 'wacki', "''", 'approach', 'began', 'infest', 'each', 'and', 'everi', 'scene', '.']
<STEMMED TOKENS>
['the', 'juda', 'priest', 'joke', 'may', 'have', 'been', 'the', 'least', 'funni', 'thing', 'i', 'have', 'ever', 'been', 'privi', 'to', ',', 'and', 'that', 'is', 'say', 'quit', 'a', 'bit', ',', 'sinc', 'it', 'doe', 'have', 'stiff', 'competit', 'from', 'all', 'the', 'other', 'joke', 'in', 'the', 'movi', '.']
<STEMMED TOKENS>
['and', 'as', 'for', 'the', 'incred', 'overus', 'of', 'the', 'pixi', "'", '``', 'debas', ',', "''", 'i', 'have', 'just', 'one', 'th

Now let's compare the stemming results to lemmatisation. For this task, NLTK provides the [class WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html) with the method lemmatize(). This method takes an argument, `pos`, that determines whether the lemmatizer is applied to nouns, verbs, adjectives or adverbs.

TODO 5: Use the WordNetLemmatizer to lemmatize the nouns in the tokenized sentences. Set the `pos` argument to 'n'. 

TODO 6: Add a second call to lemmatize() to lemmatize the verbs in the sentences as well. Set the `pos` argument to 'v'. 

How do the results compare with the Porter stemmer? 

How have the verbs in the sentences changed?

In [91]:
lemmatizer = nltk.WordNetLemmatizer() 
lemma_sents = []
for sent in tokenized_sents:
    
    ### WRITE YOUR OWN CODE HERE

    lemma_sent = [lemmatizer.lemmatize(word, pos='n') for word in sent]
    
    #######
    
    lemma_sents.append(lemma_sent)
    
    print("<LEMMATIZED TOKENS>")
    print(lemma_sent)

<LEMMATIZED TOKENS>
['Really', ',', 'when', 'it', 'come', 'down', 'to', 'it', ',', 'this', 'movie', 'is', 'just', 'not', 'funny', '.']
<LEMMATIZED TOKENS>
['Josh', 'Kornbluth', 'is', 'the', 'antithesis', 'of', 'funny', ',', 'and', 'yet', 'he', 'think', 'he', "'s", 'hilarious', '.']
<LEMMATIZED TOKENS>
['The', 'plot', 'about', 'mailing', 'the', '17', 'letter', 'wa', 'fine', 'at', 'first', ',', 'until', 'Josh', "'s", '``', 'look', 'at', 'me', ',', 'I', "'m", 'wacky', "''", 'approach', 'began', 'infesting', 'each', 'and', 'every', 'scene', '.']
<LEMMATIZED TOKENS>
['The', 'Judas', 'Priest', 'joke', 'may', 'have', 'been', 'the', 'least', 'funny', 'thing', 'I', 'have', 'ever', 'been', 'privy', 'to', ',', 'and', 'that', 'is', 'saying', 'quite', 'a', 'bit', ',', 'since', 'it', 'doe', 'have', 'stiff', 'competition', 'from', 'all', 'the', 'other', 'joke', 'in', 'the', 'movie', '.']
<LEMMATIZED TOKENS>
['And', 'a', 'for', 'the', 'incredible', 'overuse', 'of', 'the', 'Pixies', "'", '``', 'Debaser

In [92]:
for sent in tokenized_sents:
    
    ### WRITE YOUR OWN CODE HERE

    lemma_sent = [lemmatizer.lemmatize(word, pos='v') for word in sent]
    
    #######
    
    lemma_sents.append(lemma_sent)
    
    print("<LEMMATIZED TOKENS>")
    print(lemma_sent)

<LEMMATIZED TOKENS>
['Really', ',', 'when', 'it', 'come', 'down', 'to', 'it', ',', 'this', 'movie', 'be', 'just', 'not', 'funny', '.']
<LEMMATIZED TOKENS>
['Josh', 'Kornbluth', 'be', 'the', 'antithesis', 'of', 'funny', ',', 'and', 'yet', 'he', 'think', 'he', "'s", 'hilarious', '.']
<LEMMATIZED TOKENS>
['The', 'plot', 'about', 'mail', 'the', '17', 'letter', 'be', 'fine', 'at', 'first', ',', 'until', 'Josh', "'s", '``', 'look', 'at', 'me', ',', 'I', "'m", 'wacky', "''", 'approach', 'begin', 'infest', 'each', 'and', 'every', 'scene', '.']
<LEMMATIZED TOKENS>
['The', 'Judas', 'Priest', 'joke', 'may', 'have', 'be', 'the', 'least', 'funny', 'thing', 'I', 'have', 'ever', 'be', 'privy', 'to', ',', 'and', 'that', 'be', 'say', 'quite', 'a', 'bite', ',', 'since', 'it', 'do', 'have', 'stiff', 'competition', 'from', 'all', 'the', 'other', 'joke', 'in', 'the', 'movie', '.']
<LEMMATIZED TOKENS>
['And', 'as', 'for', 'the', 'incredible', 'overuse', 'of', 'the', 'Pixies', "'", '``', 'Debaser', ',', "''"

# 4. Classification

In this part, we will introduce text processing tools from the scikit-learn library. Let's start by loading the test set for the IMDB sentiment dataset.

## 4.1 Preprocessing


In [93]:
# Load the test set:
test_dataset = load_dataset(
    "imdb",
    split="test",
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset)} instances loaded")

test_dataset = np.random.choice(test_dataset, 2000, replace=False)  # we'll only use a subset of the data in this lab so that the code runs quicker


Test dataset with 25000 instances loaded


The next step is to tokenise the text of each tweet and convert it to a bag of words, ready for input to a classifier. 
To do this, we will use the scikit-learn library. 

To extract a bag of words, we can use the CountVectorizer class ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).
This class outputs the bag of words as a feature vector, where the length of the vector is equal to the size of the vocabulary, and the values are the counts of each words in a document. 

Run the code below to obtain feature vectors for the training and test samples:

In [94]:
train_text = [sample["text"] for sample in train_dataset]
train_label = [sample["label"] for sample in train_dataset]
test_text = [sample["text"] for sample in test_dataset]
test_label =  [sample["label"] for sample in test_dataset]

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

# CountVectorizer can do its own tokenization, but for consistency we want to
# carry on using WordNetTokenizer. 
vectorizer = CountVectorizer(tokenizer=word_tokenize)  # construct the vectorizer

vectorizer.fit(train_text)  # Learn the vocabulary
X_train = vectorizer.transform(train_text)  # extract training set bags of words
X_test = vectorizer.transform(test_text)  # extract test set bags of words

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


The fit() method sets the vectorizer up by extracting a vocabulary from some text data. 

QUESTION: Why do we fit the CountVectorizer on the training set only?

The vectorizer stores the vocabulary as a dictionary that maps a token to its index in the feature vector. The code below looks up the indexes of some example words:

In [96]:
import reprlib

vocabulary = vectorizer.vocabulary_
print(vocabulary['the'])
print(vocabulary['horse'])
print(vocabulary['smile'])
print(vocabulary['happy'])
print(vocabulary['hate'])

print(f'Vocabulary size = {len(vocabulary)}')

44198
21920
40786
20616
20799
Vocabulary size = 49586


## 4.2 Naive Bayes Classifier

The code above has obtained the feature vectors and lists of labels. The data is now ready for use
with scikit-learn's classifiers.

Scikit-learn contains several different variants of naïve Bayes for different kinds of data. For our bag of words data, we need to use the [MultinomialNB class](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB).


TODO 7: Look at the documentation for MultinomalNB and write code to train a NB classifier using `X_train` and `train_label`.

In [97]:
# WRITE YOUR CODE HERE
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, train_label)


MultinomialNB()

Now we have a trained model, we would like to evaluate its performance on some test data. 

TODO 8: Refer to the documentation again and predict the labels for the test set. Use `X_test` as the inputs to the classifier.

In [98]:
# WRITE YOUR CODE HERE
predict = clf.predict(X_test)


We can compute metrics for classifier performance using [scikit-learn's metrics libary](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules). A useful function for multi-class classification (when there are more than two classes) is the [classification report function](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report).

TODO 9: Refer again to the documentation, and compute accuracy, precision, recall and F1 scores on the test set. 

In [99]:
# WRITE YOUR CODE HERE
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))


              precision    recall  f1-score   support

           0       0.75      0.89      0.81       987
           1       0.87      0.71      0.78      1013

    accuracy                           0.80      2000
   macro avg       0.81      0.80      0.80      2000
weighted avg       0.81      0.80      0.80      2000



Now, let's examine the classifier that we learned. You may wish to refer back to the slides on naïve Bayes classifiers or to [Jurafsky and Martin's textbook](https://web.stanford.edu/~jurafsky/slp3/4.pdf) to see how the implementation relates to the equations. 

Previously, we trained a MultinomialNB classifier. The trained classifier object stores all the probabilities that it learned during training, which are needed to make predictions. The log of the likelihoods of each word given the class are represented by the attribute `feature_log_prob_`. So, if your classifier object is named `classifier`, you can access the log likelihoods with `classifier.feature_log_prob_`.

TODO 10: Print out the likelihood of the words 'happy' and 'hate' in each class. Hint: look up the index of the chosen words in `vocabulary`. The rows of `feature_log_prob` correspond to classes, and the columns to words.

In [102]:
import numpy as np

### CHANGE THE NAME OF THE CLASSIFIER VARIABLE BELOW TO USE YOUR TRAINED CLASSIFIER
feat_likelihoods = np.exp(clf.feature_log_prob_)  # Use exponential to convert the logs back to probabilities
###

# WRITE YOUR CODE HERE
print(feat_likelihoods[:, vocabulary['happy']])
print(feat_likelihoods[:, vocabulary['hate']])

[8.75214068e-05 1.26589547e-04]
[9.92780137e-05 7.43876720e-05]


The sentiment classes are negative (0) and positive (1). 

QUESTION: According to the feature log likelihoods, which class has the strongest association with 'happy' and with 'hate'?

A key part of evaluating a classifier is investigating the errors it makes to better understand its limitations. 

TODO 11: Complete the code below to print out some misclassified reviews along with their predicted and true labels. Can you see any reasons why a NB classiier would make mistakes with these instances?

In [ ]:
error_indexes = y_test_pred != test_label  # compare predictions to gold labels

# get the text of reviews where the classifier made an error:
reviews_err = np.array(test_text)[error_indexes]

# WRITE YOUR CODE HERE



## 4.3. Logistic Regression Classifier

Another simple, linear classifier is logistic regression. This classifier does not rely on the conditional independence assumption, so can better model features that are highly correlated with each other. Scikit-learn provides the [logisticRegression class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), which has a very similar interface to the naïve Bayes classifier.

TODO 12: Train a logistic regression classifier, referring to the scikit-learn documentation as required.

In [ ]:
# WRITE YOUR CODE HERE



In [ ]:
# WRITE YOUR CODE HERE



In [ ]:
# WRITE YOUR CODE HERE


QUESTION: How does the performance of logistic regression compare with naïve Bayes?

The logistic regression classifier works by learning a weight for each feature that indicates its importance in predicting a class. These weights are stored in the `coef_` attribute of the LogisticRegression object. For the binary case, it provides a single weight per feature. This allows it to compute the probability of the positive class as _p_, then compute the probability of the negative class as _1-p_.  For multi-class problems, there is a row of weights corresponding to each class, and columns corresponding to words in the vocabulary. 

TODO 13: Print out the weights for 'happy' and 'hate'.

In [ ]:
### WRITE YOUR CODE HERE



QUESTION: Are the weights what you would expect to see?

The code below prints out the words with the highest weights for the positive class. We use numpy's `argsort` function to get the indexes of the sorted weights. Run the code below to show the result: 

In [ ]:
n_feats_to_show = 10

# Flip the index so that values are keys and keys are values:
keys = vectorizer.vocabulary_.values()
values = vectorizer.vocabulary_.keys()
vocab_inverted = dict(zip(keys, values))

strongest_idxs = np.argsort(classifier.coef_[0])[-n_feats_to_show:]

for idx in strongest_idxs:
    print(f'{vocab_inverted[idx]} with weight {classifier.coef_[0][idx]}')

TODO 14: Use the same code as for naïve Bayes to print out examples of misclassified tweets and their labels. Hint: you should be able to copy and paste your code from above :) 

In [ ]:
error_indexes = y_test_pred != test_label  # compare predictions to gold labels

# get the text of tweets where the classifier made an error:
reviews_err = np.array(test_text)[error_indexes]

# WRITE YOUR CODE HERE




# 5. OPTIONAL: Lemmatization and N-grams

You only need to do this section if you finish the previous sections before the end of the lab.

In the previous lab, we tried out lemmatization. This is useful for reducing the size of the vocabulary. Could it help us here?

To apply lemmatization, we have to go back to the CountVectorizer and define a new tokenizer class that will carry out the extra step of lemmatization. Run the code below to test this out:

In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer(object):  # this 'tokenizer' will also do additional preprocessing steps, namely, lemmatize verbs and adjectives
    
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        
    def __call__(self, tweets):
        return [self.wnl.lemmatize(self.wnl.lemmatize(tok, pos='v'), pos='a') for tok in word_tokenize(tweets)]
    
vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())

vectorizer.fit(train_text)
X_train = vectorizer.transform(train_text)
X_test = vectorizer.transform(test_text)

# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

In [ ]:
print(f'Vocabulary size: {len(vectorizer.vocabulary_)}')

TODO 15: Now, repeat your training of the NB classifier using the new features, and compare its performance with the previous classifers.

In [ ]:
### WRITE YOUR OWN CODE HERE



QUESTION: Did lemmatization bring about any improvements on this dataset?

The bag of words is a very simple representation of the text that does not capture enough information to make accurate sentiment classifications. Another way to improve it could be to use bigrams instead of single words as our features. Bigrams are pairs of words that occur one after another in the text. Bigrams are a kind of 'n-gram', where 'n=2'. 

To extract bigrams, we again modify our CountVectorizer. This class has a parameter `ngram_range`, which determines the range of sizes of n-grams the vectorizer will include. If we set `ngram_range=(1,1)` we have our standard bag of words. If we set it to `ngram_range=(2,2)`, we use bigrams instead. Choosing If we set `ngram_range=(1,2)` will use both single tokens (unigrams) and bigrams.

TODO 16: Create a new CountVectorizer that extracts bigram features as well as unigrams (single tokens).

In [ ]:
### WRITE YOUR CODE HERE



###

print(f'Vocabulary size: {len(vectorizer.vocabulary_)}')

# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

TODO 17: Now, repeat your training of the logistic regression or naïve Bayes classifier using the new features, and compare its performance with the previous classifers.

In [ ]:
### WRITE YOUR OWN CODE HERE



QUESTION: Do bigrams improve performance on this dataset?

# 6. Optional: Lexicon Features

You only need to do this part if you finish the other parts before the end of the lab session. 

The NLTK library contains sentiment lexicons, which are lists of words with negative or positive connotations. 

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyser = SentimentIntensityAnalyzer()

Let's have a look at the sentiment scores for some words in the lexicon by running the code below. What do the scores mean and why do some words have no score?

In [ ]:
testwords = ['happy', 'wonderful', 'horrible', 'boring', 'tablecloth', 'not']

for word in testwords:
    if word in analyser.lexicon:
        print(f'{word}: {analyser.lexicon[word]}')
    else:
        print(f'{word}: NOT IN LEXICON')

Now we would like to use this function to compute counts of all positive and negative words. Let's start by recording whether the words in our vocabulary are positive or negative:

In [ ]:
# Print out some of the features in the vocabulary:
print(list(vectorizer.vocabulary_)[:20])

vocabulary = vectorizer.vocabulary_

lex_pos_scores = np.zeros((1, len(vocabulary)))
lex_neg_scores = np.zeros((1, len(vocabulary)))

# get the Vader lexicon scores for each word in our vocabulary
for i, term in enumerate(vocabulary):
    if term in analyser.lexicon and analyser.lexicon[term] > 0:
        lex_pos_scores[0, i] = 1
    elif term in analyser.lexicon and analyser.lexicon[term] < 0:
        lex_neg_scores[0, i] = 1

Now let's compute the counts of positive and negative words in the dataset:

In [ ]:
# Compute the scores for each instance in the data set. 

# Multiply the lexicon scores by the feature vectors, then sum over the 
# vocabulary to get the total positive and total negative counts:
lex_pos_train = np.sum(X_train.multiply(lex_pos_scores), axis=1)
lex_pos_test = np.sum(X_test.multiply(lex_pos_scores), axis=1)

lex_neg_train = np.sum(X_train.multiply(lex_neg_scores), axis=1)
lex_neg_test = np.sum(X_test.multiply(lex_neg_scores), axis=1)

Finally, we can append the counts to the feature vector and treat them as extra features:

In [ ]:
from scipy.sparse import hstack

X_train = hstack((X_train, lex_pos_train, lex_neg_train))
X_test = hstack((X_test, lex_pos_test, lex_neg_test))

TO-DO 18: Use the new X_train and X_test feature vectors to train and evaluate your classifier. 
Does adding the lexicon features improve performance?

In [ ]:
### WRITE YOUR OWN CODE HERE

